<a href="https://colab.research.google.com/github/DionKimmm/2019_Summer_DL_Prof_Seok/blob/master/190801_Fashion_MNIST_0_94_accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip fashion-mnist.zip

unzip:  cannot find or open fashion-mnist.zip, fashion-mnist.zip.zip or fashion-mnist.zip.ZIP.


In [2]:
!rm fashion-mnist.zip

rm: cannot remove 'fashion-mnist.zip': No such file or directory


# 참고 블로그

https://www.pyimagesearch.com/2019/02/11/fashion-mnist-with-keras-and-deep-learning/

# 모델

* minivggnet.py
* (CONV => RELU => BN(배치정규화)) * 2 => POOL
* => Fully Connected => Softmax

# 단계

## 필요한 라이브러리 불러오기

### 플롯을 백그라운드에서 실행 가능하게 하는 코드

In [0]:
import matplotlib
matplotlib.use("Agg")

* 패션 MNIST 셋은 케라스안에도 있습니다.

In [4]:
# import the necessary packages
from pyimagesearch.minivggnet import MiniVGGNet
from sklearn.metrics import classification_report
from keras.optimizers import SGD
from keras.datasets import fashion_mnist
from keras.utils import np_utils
from keras import backend as K
from imutils import build_montages
import matplotlib.pyplot as plt
import numpy as np
import cv2

Using TensorFlow backend.


## 학습, 테스트, 레이블 데이터 불러오기

In [0]:
NUM_EPOCHS = 50
INIT_LR = 1e-2
BS = 32 # batch size

케라스 백엔드 이미지 데이터 포맷에 channels_first와 last가 있고, 그에 맞게 re-order 한다고 합니다.

In [6]:
# grab the Fashion MNIST dataset (if this is your first time running
# this the dataset will be automatically downloaded)
print("[INFO] loading Fashion MNIST...")
((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()
print("Finish!")
# if we are using "channels first" ordering, then reshape the design
# matrix such that the matrix is:
# 	num_samples x depth x rows x columns
if K.image_data_format() == "channels_first":
	trainX = trainX.reshape((trainX.shape[0], 1, 28, 28))
	testX = testX.reshape((testX.shape[0], 1, 28, 28))
 # otherwise, we are using "channels last" ordering, so the design
 # matrix shape should be: num_samples x rows x columns x depth
else:
	trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
	testX = testX.reshape((testX.shape[0], 28, 28, 1))

[INFO] loading Fashion MNIST...
Finish!


### preprocess

In [0]:
# scale data to the range of [0, 1]
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0
 
# one-hot encode the training and testing labels
trainY = np_utils.to_categorical(trainY, 10)
testY = np_utils.to_categorical(testY, 10)
 
# initialize the label names
labelNames = ["top", "trouser", "pullover", "dress", "coat",
	"sandal", "shirt", "sneaker", "bag", "ankle boot"]

## 학습

* SGD를 옵티마이저로 사용합니다.

In [8]:
# initialize the optimizer and model
print("[INFO] compiling model...")
opt = SGD(lr=INIT_LR, momentum=0.9, decay=INIT_LR / NUM_EPOCHS)
model = MiniVGGNet.build(width=28, height=28, depth=1, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
 
# train the network
print("[INFO] training model...")
H = model.fit(trainX, trainY,
	validation_data=(testX, testY),
	batch_size=BS, epochs=NUM_EPOCHS)

W0801 05:32:56.874039 139851620988800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0801 05:32:56.875864 139851620988800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0801 05:32:56.877924 139851620988800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



[INFO] compiling model...


W0801 05:32:56.913070 139851620988800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0801 05:32:56.914053 139851620988800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0801 05:32:57.628007 139851620988800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0801 05:32:57.772978 139851620988800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0801 05:32:57.781141 13985162

[INFO] training model...
Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 19s 321us/step - loss: 0.5422 - acc: 0.8191 - val_loss: 0.4370 - val_acc: 0.8396
Epoch 2/50
60000/60000 [==============================] - 17s 277us/step - loss: 0.3477 - acc: 0.8765 - val_loss: 0.3692 - val_acc: 0.8704
Epoch 3/50
60000/60000 [==============================] - 16s 275us/step - loss: 0.2987 - acc: 0.8931 - val_loss: 0.3126 - val_acc: 0.8857
Epoch 4/50
60000/60000 [==============================] - 17s 277us/step - loss: 0.2689 - acc: 0.9023 - val_loss: 0.2422 - val_acc: 0.9116
Epoch 5/50
60000/60000 [==============================] - 17s 275us/step - loss: 0.2531 - acc: 0.9091 - val_loss: 0.2239 - val_acc: 0.9204
Epoch 6/50
60000/60000 [==============================] - 16s 275us/step - loss: 0.2383 - acc: 0.9143 - val_loss: 0.2155 - val_acc: 0.9204
Epoch 7/50
60000/60000 [==============================] - 16s 274us/step - loss: 0.2256 - a

## 테스트

* 테스트하고 결과값을 이미지로 저장합니다.

In [9]:
# make predictions on the test set
preds = model.predict(testX)
 
# show a nicely formatted classification report
print("[INFO] evaluating network...")
print(classification_report(testY.argmax(axis=1), preds.argmax(axis=1),
	target_names=labelNames))
 
# plot the training loss and accuracy
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot_epoch_50.png")

[INFO] evaluating network...
              precision    recall  f1-score   support

         top       0.89      0.91      0.90      1000
     trouser       0.99      0.99      0.99      1000
    pullover       0.91      0.92      0.92      1000
       dress       0.94      0.95      0.94      1000
        coat       0.91      0.90      0.91      1000
      sandal       0.99      0.99      0.99      1000
       shirt       0.84      0.81      0.82      1000
     sneaker       0.96      0.98      0.97      1000
         bag       0.99      0.99      0.99      1000
  ankle boot       0.98      0.97      0.98      1000

    accuracy                           0.94     10000
   macro avg       0.94      0.94      0.94     10000
weighted avg       0.94      0.94      0.94     10000



## To Visualize

* 를 하려고 했는데 cv2.imshow() is disabled in Colab 이라고 합니다.

In [0]:
# # initialize our list of output images
# images = []
 
# # randomly select a few testing fashion items
# for i in np.random.choice(np.arange(0, len(testY)), size=(16,)):
# 	# classify the clothing
# 	probs = model.predict(testX[np.newaxis, i])
# 	prediction = probs.argmax(axis=1)
# 	label = labelNames[prediction[0]] 
# 	# extract the image from the testData if using "channels_first"
# 	# ordering
# 	if K.image_data_format() == "channels_first":
# 		image = (testX[i][0] * 255).astype("uint8")
# 	# otherwise we are using "channels_last" ordering
# 	else:
# 		image = (testX[i] * 255).astype("uint8")
# 	# initialize the text label color as green (correct)
# 	color = (0, 255, 0)
# 	# otherwise, the class label prediction is incorrect
# 	if prediction[0] != np.argmax(testY[i]):
# 		color = (0, 0, 255)
# 	# merge the channels into one image and resize the image from
# 	# 28x28 to 96x96 so we can better see it and then draw the
# 	# predicted label on the image
# 	image = cv2.merge([image] * 3)
# 	image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)
# 	cv2.putText(image, label, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.75,
# 		color, 2)
# 	# add the image to our list of output images
# 	images.append(image)
# # construct the montage for the images
# montage = build_montages(images, (96, 96), (4, 4))[0]
 
# # show the output montage
# cv2.imshow("Fashion MNIST", montage)
# cv2.waitKey(0)